In [ ]:
import os 
import json
os.environ["HF_HOME"] = "/your/path/hf_cache"
from datasets import load_dataset 

    

In [ ]:
# Loading the annotations of COCO dataset
# Using the categories in a specific annotation (COCO-Stuff)
with open('COCO-Stuff/stuff_val.json', 'r') as f:
    data = json.load(f)

In [ ]:
# Use the number of categories to filter the images
# The categories are: wood, stone, metal, paper, brick
wood_ids, stone_ids, metal_ids, paper_ids, brick_ids = [], [], [], [], []
for anno in data['annotations']:
    if anno['category_id'] == 182 or anno['category_id'] == 177 or anno['category_id'] == 118:
        wood_ids.append(anno['image_id'])
    elif anno['category_id'] == 116 or anno['category_id'] == 162 or anno['category_id'] == 175:
        stone_ids.append(anno['image_id'])
    elif anno['category_id'] == 132:
        metal_ids.append(anno['image_id'])
    elif anno['category_id'] == 139:
        paper_ids.append(anno['image_id'])
    elif anno['category_id'] == 171:
        brick_ids.append(anno['image_id'])

In [ ]:
mix_ids = []
for id in wood_ids:
    if id in stone_ids or id in metal_ids or id in paper_ids or id in brick_ids:
        mix_ids.append(id)
for id in stone_ids:
    if id in metal_ids or id in paper_ids or id in brick_ids:
        mix_ids.append(id)
for id in metal_ids:
    if id in paper_ids or id in brick_ids:
        mix_ids.append(id)
for id in paper_ids:
    if id in brick_ids:
        mix_ids.append(id)
        
print(len(mix_ids))
print(len(wood_ids), len(stone_ids), len(metal_ids), len(paper_ids), len(brick_ids))

In [ ]:
with open('material_ids.txt', 'w') as f:
    f.write('wood_ids:\n')
    for id in reversed(wood_ids):
        if id in mix_ids:
            wood_ids.remove(id)
            continue
        f.write(str(id) + '\n')
    f.write('stone_ids:\n')
    for id in reversed(stone_ids):
        if id in mix_ids:
            stone_ids.remove(id)
            continue
        f.write(str(id) + '\n')
    f.write('metal_ids:\n')
    for id in reversed(metal_ids):
        if id in mix_ids:
            metal_ids.remove(id)
            continue
        f.write(str(id) + '\n')
    f.write('paper_ids:\n')
    for id in reversed(paper_ids):
        if id in mix_ids:
            paper_ids.remove(id)
            continue
        f.write(str(id) + '\n')
    f.write('brick_ids:\n')
    for id in reversed(brick_ids):
        if id in mix_ids:
            brick_ids.remove(id)
            continue
        f.write(str(id) + '\n')
    f.write('mix_ids:\n')

In [ ]:
with open("bad_ids.txt" ,'r') as f:
    material_ids = f.readlines()
    for i in range(len(material_ids)):
        id = material_ids[i].strip()
        if id in wood_ids:
            wood_ids.remove(id)
        elif id in stone_ids:
            stone_ids.remove(id)
        elif id in metal_ids:
            metal_ids.remove(id)
        elif id in paper_ids:
            paper_ids.remove(id)
        elif id in brick_ids:
            brick_ids.remove(id)

In [ ]:
# materials = ['wood', 'stone', 'brick', 'paper', 'metal']
import random
# Generating negative samples
# The negative samples are generated based on the positive samples
nega_map_wood, nega_map_stone, nega_map_metal, nega_map_paper, nega_map_brick = {}, {}, {}, {}, {}
for i in wood_ids:
    nega_ids =[]
    nega_ids.extend(stone_ids)
    nega_ids.extend(metal_ids)
    nega_ids.extend(paper_ids)
    nega_ids.extend(brick_ids)
    nega_ids = random.sample(nega_ids, 99)
    nega_map_wood[str(i)] = nega_ids
for i in stone_ids:
    nega_ids =[]
    nega_ids.extend(wood_ids)
    nega_ids.extend(metal_ids)
    nega_ids.extend(paper_ids)
    nega_ids = random.sample(nega_ids, 99)
    nega_map_stone[str(i)] = nega_ids
for i in metal_ids:
    nega_ids =[]
    nega_ids.extend(wood_ids)
    nega_ids.extend(stone_ids)
    nega_ids.extend(paper_ids)
    nega_ids.extend(brick_ids)
    nega_ids = random.sample(nega_ids, 99)
    nega_map_metal[str(i)] = nega_ids
for i in paper_ids:
    nega_ids =[]
    nega_ids.extend(wood_ids)
    nega_ids.extend(stone_ids)
    nega_ids.extend(metal_ids)
    nega_ids.extend(brick_ids)
    nega_ids = random.sample(nega_ids, 99)
    nega_map_paper[str(i)] = nega_ids
for i in brick_ids:
    nega_ids =[]
    nega_ids.extend(wood_ids)
    nega_ids.extend(metal_ids)
    nega_ids.extend(paper_ids)
    nega_ids = random.sample(nega_ids, 99)
    nega_map_brick[str(i)] = nega_ids

In [ ]:
def generate_sample(material, image_ids):
    # Generate a sample for the given material and image IDs
    
    text = f"Find me an everyday image showing some object or surface made of {material}."

    sample = {
        "qry_text": f"{text}",
        "qry_img_path": "",
        "tgt_text": "<|image_1|> Represent the given image.",
        "tgt_img_path": f"{image_ids}"
    }
    return sample

In [ ]:
final = []
for mapping in nega_map_wood:
    image_ids = [int(mapping)] + nega_map_wood[mapping]
    material = 'wood'
    # Generate a sample for the given people count and image IDs
    sample = generate_sample(material, image_ids)
    final.append(sample)
for mapping in nega_map_stone:
    image_ids = [int(mapping)] + nega_map_stone[mapping]
    material = 'stone'
    # Generate a sample for the given people count and image IDs
    sample = generate_sample(material, image_ids)
    final.append(sample)
for mapping in nega_map_metal:
    image_ids = [int(mapping)] + nega_map_metal[mapping]
    material = 'metal'
    # Generate a sample for the given people count and image IDs
    sample = generate_sample(material, image_ids)
    final.append(sample)
for mapping in nega_map_paper:
    image_ids = [int(mapping)] + nega_map_paper[mapping]
    material = 'paper'
    # Generate a sample for the given people count and image IDs
    sample = generate_sample(material, image_ids)
    final.append(sample)
for mapping in nega_map_brick:
    image_ids = [int(mapping)] + nega_map_brick[mapping]
    material = 'brick'
    # Generate a sample for the given people count and image IDs
    sample = generate_sample(material, image_ids)
    final.append(sample)

In [ ]:
# Saving the final samples to a JSON file
os.makedirs("material", exist_ok=True)
output_file = os.path.join("material", "COCOStuff_material_retrieval.json")
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(final, f, ensure_ascii=False, indent=4)